In [ ]:
import src.dataPipeline as dataPipeline
import importlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
# Evaluating the model
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error,r2_score ,make_scorer
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

importlib.reload(dataPipeline)


In [ ]:
def calculate_metrics(X_train, y_test, y_pred):
    n = len(y_test)  # Number of observations
    k = X_train.shape[1]  # Number of predictors
    r2 = round(r2_score(y_test, y_pred), 4)
    R2_adjusted = round(1 - (1 - r2) * (n - 1) / (n - k - 1), 4)
    mape = round(mean_absolute_percentage_error(y_test, y_pred) * 100, 4)
    return r2, R2_adjusted, mape

In [ ]:
def manual_cv_score(X, y, cv, model):
    fold_train_mapes = []
    fold_test_mapes = []

    # Manual CV loop
    for train_idx, test_idx in cv.split(X):
        # Split the data for this fold
        X_tr, X_te = X.iloc[train_idx], X.iloc[test_idx]
        y_tr, y_te = y.iloc[train_idx], y.iloc[test_idx]

        # Fit the model
        model.fit(X_tr, y_tr)

        # Predict on training fold
        y_tr_pred = model.predict(X_tr)
        fold_train_mapes.append(mean_absolute_percentage_error(y_tr, y_tr_pred))

        # Predict on test fold
        y_te_pred = model.predict(X_te)
        fold_test_mapes.append(mean_absolute_percentage_error(y_te, y_te_pred))

    # Calculate mean & std for train/test MAPE across folds
    train_mape_mean = np.mean(fold_train_mapes) * 100
    train_mape_std  = np.std(fold_train_mapes)  * 100

    test_mape_mean  = np.mean(fold_test_mapes)  * 100
    test_mape_std   = np.std(fold_test_mapes)   * 100

    return train_mape_mean, train_mape_std, test_mape_mean, test_mape_std

In [ ]:
dp = dataPipeline.DataPipeline()
df = dp.runPipeline(
    filePath="../data/immo_data_202208_v2.csv",
    imputer=None,
    normalizeAndStandardize= False,
    get_dummies = False
)

In [ ]:
#show all columns
pd.set_option('display.max_columns', None)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
#Categorical columns
cat_col =["Availability","type_unified","region_group"]

In [ ]:
df[cat_col] = df[cat_col].astype('category')

In [ ]:
df[cat_col].isna().sum()

In [ ]:
df.head()

In [ ]:
df.loc[df['Space extracted'] < 5, 'Space extracted'] = np.nan
df.loc[df["Floor"] >= 41, "Floor"] = np.nan

#Filling Floor for House types with zeros
house_types = [
    'detached-house', 'villa', 'semi-detached-house', 'terrace-house',
    'chalet', 'farmhouse', 'rustico', 'castle', 'detached-secondary-suite'
]
df.loc[
    (df['type_unified'].isin(house_types)) & (df['Floor'].isna()),
    'Floor'
] = 0
#Fill na with 0
df["detail_responsive#surface_usable"] = df["detail_responsive#surface_usable"].fillna(0)
df["Number of floors:"] = df["Number of floors:"].fillna(1)
df["Plot_area_unified"] = df["Plot_area_unified"].fillna(0)

In [ ]:
df.head()

In [ ]:
#drop column Availability
df = df.drop(columns=["Availability"])

In [ ]:
# Define categorical and numerical columns
cat_col = ["type_unified", "region_group"]
num_col = [col for col in df.columns if col not in cat_col + ["price_cleaned"]]

# Preprocessing for numerical data
numerical_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=5)),
    ('scaler', StandardScaler())
])

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessors in a column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, num_col),
        ('cat', categorical_transformer, cat_col)
    ]
)

# Random Forest baseline model

In [ ]:
rf_baseline = RandomForestRegressor(
    n_estimators=100,
    criterion='squared_error',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features=1.0,
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    bootstrap=True,
    oob_score=False,
    n_jobs=None,
    random_state=42,
    verbose=0,
    warm_start=False,
    ccp_alpha=0.0,
    max_samples=None,
    monotonic_cst=None
)

In [ ]:
rf_estimater_10 = RandomForestRegressor(
    n_estimators=10,
    criterion='squared_error',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features=1.0,
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    bootstrap=True,
    oob_score=False,
    n_jobs=None,
    random_state=42,
    verbose=0,
    warm_start=False,
    ccp_alpha=0.0,
    max_samples=None,
    monotonic_cst=None
)

In [ ]:
# Create and combine preprocessing and modeling in a pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', rf_baseline)
])

# Separate target and features
X = df.drop(columns=["price_cleaned"])
y = df["price_cleaned"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)
pipeline.fit(X_train, y_train)

In [ ]:
#caluclate mape on train data
y_pred = pipeline.predict(X_train)
r2, R2_adjusted, mape = calculate_metrics(X_train, y_train, y_pred)
print(f"R2 {r2},Adjusted R^2:{R2_adjusted}, MAPE:{mape}%")

In [ ]:
#caluclate mape on test data
y_pred = pipeline.predict(X_test)
r2, R2_adjusted, mape=calculate_metrics(X_train, y_test, y_pred)
print(f"R2 {r2},Adjusted R^2:{R2_adjusted}, MAPE:{mape}%")

# Hyperparameter tuning: Estimators

In [ ]:
# Suppose we have these
estimators = [10, 20, 50, 70, 100, 200]
cv = KFold(n_splits=5, shuffle=True, random_state=42)

X_train_preprocessed = preprocessor.fit_transform(X_train)
X_train_preprocessed = pd.DataFrame(X_train_preprocessed)


train_mape_scores_mean = []
train_mape_scores_std = []
test_mape_scores_mean = []
test_mape_scores_std = []

In [ ]:
for estimator in tqdm(estimators):
    rf_estimator = RandomForestRegressor(
        n_estimators=estimator,
        criterion='squared_error',
        random_state=42,
        n_jobs=-1
    )
    pipeline = Pipeline(steps=[('model', rf_estimator)])

    # For storing MAPE per fold
    fold_train_mapes = []
    fold_test_mapes = []

    # Manual CV loop
    for train_idx, test_idx in cv.split(X_train_preprocessed):
        X_tr, X_te = X_train_preprocessed.iloc[train_idx], X_train_preprocessed.iloc[test_idx]
        y_tr, y_te = y_train.iloc[train_idx], y_train.iloc[test_idx]

        pipeline.fit(X_tr, y_tr)

        # Predict on training fold
        y_tr_pred = pipeline.predict(X_tr)
        fold_train_mapes.append(mean_absolute_percentage_error(y_tr, y_tr_pred))

        # Predict on test fold
        y_te_pred = pipeline.predict(X_te)
        fold_test_mapes.append(mean_absolute_percentage_error(y_te, y_te_pred))

    # Average across folds
    train_mape_mean = np.mean(fold_train_mapes) * 100
    train_mape_std = np.std(fold_train_mapes) * 100

    test_mape_mean = np.mean(fold_test_mapes) * 100
    test_mape_std = np.std(fold_test_mapes) * 100

    # Store results
    train_mape_scores_mean.append(train_mape_mean)
    train_mape_scores_std.append(train_mape_std)
    test_mape_scores_mean.append(test_mape_mean)
    test_mape_scores_std.append(test_mape_std)

    print(
        f"Estimator: {estimator}, "
        f"Train MAPE: {train_mape_mean:.2f}% (±{train_mape_std:.2f}), "
        f"Test MAPE: {test_mape_mean:.2f}% (±{test_mape_std:.2f})"
    )

In [ ]:
# Plot the MAPE scores
plt.figure(figsize=(8, 5))
plt.errorbar(estimators, train_mape_scores_mean, yerr=train_mape_scores_std, label='Train MAPE', marker='o', capsize=5)
plt.errorbar(estimators, test_mape_scores_mean, yerr=test_mape_scores_std, label='Test MAPE', marker='o', capsize=5)
plt.xlabel('Number of Estimators')
plt.ylabel('MAPE (%)')
plt.title('Train vs Test MAPE Scores for Random Forest')
plt.legend()
plt.grid(True)
plt.show()



# Hyperparameter tuning: Max Depth

In [ ]:
# Suppose we have these
depths = [7, 10, 15, 20, 25, 30, 35]
cv = KFold(n_splits=5, shuffle=True, random_state=42)

X_train_preprocessed = preprocessor.fit_transform(X_train)
X_train_preprocessed = pd.DataFrame(X_train_preprocessed)

train_mape_scores_mean = []
train_mape_scores_std = []
test_mape_scores_mean = []
test_mape_scores_std = []
for depth in tqdm(depths):
    rf_estimator = RandomForestRegressor(
        n_estimators=70,
        max_depth=depth,
        criterion='squared_error',
        random_state=42,
        n_jobs=-1
    )
    pipeline = Pipeline(steps=[('model', rf_estimator)])

    # For storing MAPE per fold
    fold_train_mapes = []
    fold_test_mapes = []

    # Manual CV loop
    for train_idx, test_idx in cv.split(X_train_preprocessed):
        X_tr, X_te = X_train_preprocessed.iloc[train_idx], X_train_preprocessed.iloc[test_idx]
        y_tr, y_te = y_train.iloc[train_idx], y_train.iloc[test_idx]

        pipeline.fit(X_tr, y_tr)

        # Predict on training fold
        y_tr_pred = pipeline.predict(X_tr)
        fold_train_mapes.append(mean_absolute_percentage_error(y_tr, y_tr_pred))

        # Predict on test fold
        y_te_pred = pipeline.predict(X_te)
        fold_test_mapes.append(mean_absolute_percentage_error(y_te, y_te_pred))

    # Average across folds
    train_mape_mean = np.mean(fold_train_mapes) * 100
    train_mape_std = np.std(fold_train_mapes) * 100

    test_mape_mean = np.mean(fold_test_mapes) * 100
    test_mape_std = np.std(fold_test_mapes) * 100

    # Store results
    train_mape_scores_mean.append(train_mape_mean)
    train_mape_scores_std.append(train_mape_std)
    test_mape_scores_mean.append(test_mape_mean)
    test_mape_scores_std.append(test_mape_std)

    print(
        f"Estimator: {depth}, "
        f"Train MAPE: {train_mape_mean:.2f}% (±{train_mape_std:.2f}), "
        f"Test MAPE: {test_mape_mean:.2f}% (±{test_mape_std:.2f})"
    )

In [ ]:
# Plot the MAPE scores
plt.figure(figsize=(8, 5))
plt.errorbar(depths, train_mape_scores_mean, yerr=train_mape_scores_std, label='Train MAPE', marker='o', capsize=5)
plt.errorbar(depths, test_mape_scores_mean, yerr=test_mape_scores_std, label='Test MAPE', marker='o', capsize=5)
plt.xlabel('Max Depth')
plt.ylabel('MAPE (%)')
plt.title('Train vs Test MAPE Scores for Random Forest')
plt.legend()
plt.grid(True)
plt.show()

# Hyperparameter tuning: Min Samples Split

In [ ]:
min_samples_splits  = [2, 3, 4, 5]
cv = KFold(n_splits=5, shuffle=True, random_state=42)

X_train_preprocessed = preprocessor.fit_transform(X_train)
X_train_preprocessed = pd.DataFrame(X_train_preprocessed)

# Prepare lists to store results
train_mape_scores_mean = []
train_mape_scores_std = []
test_mape_scores_mean = []
test_mape_scores_std = []

# Loop over different depths
for split in tqdm(min_samples_splits):
    # Define your estimator with the current max_depth
    rf_estimator = RandomForestRegressor(
        n_estimators=70,
        max_depth=25,
        min_samples_split=split,
        criterion='squared_error',
        random_state=42,
        n_jobs=-1
    )

    # Build a Pipeline (you can add more steps if needed)
    pipeline = Pipeline(steps=[('model', rf_estimator)])

    # Call the function to perform manual CV and get MAPE metrics
    train_mape_mean, train_mape_std, test_mape_mean, test_mape_std = manual_cv_score(
        X=X_train_preprocessed,
        y=y_train,
        cv=cv,
        model=pipeline
    )

    # Store results
    train_mape_scores_mean.append(train_mape_mean)
    train_mape_scores_std.append(train_mape_std)
    test_mape_scores_mean.append(test_mape_mean)
    test_mape_scores_std.append(test_mape_std)

    # Print progress
    print(
        f"Max split: {split}, "
        f"Train MAPE: {train_mape_mean:.2f}% (±{train_mape_std:.2f}), "
        f"Test MAPE: {test_mape_mean:.2f}% (±{test_mape_std:.2f})"
    )

In [ ]:
# Plot the MAPE scores
plt.figure(figsize=(8, 5))
plt.errorbar(depths, train_mape_scores_mean, yerr=train_mape_scores_std, label='Train MAPE', marker='o', capsize=5)
plt.errorbar(depths, test_mape_scores_mean, yerr=test_mape_scores_std, label='Test MAPE', marker='o', capsize=5)
plt.xlabel('Max Depth')
plt.ylabel('MAPE (%)')
plt.title('Train vs Test MAPE Scores for Random Forest')
plt.legend()
plt.grid(True)
plt.show()

# Hyperparameter tuning: Max Features


In [ ]:
max_features = [1.0, 0.5, 0.3, 'sqrt', 'log2']

cv = KFold(n_splits=5, shuffle=True, random_state=42)

X_train_preprocessed = preprocessor.fit_transform(X_train)
X_train_preprocessed = pd.DataFrame(X_train_preprocessed)

# Prepare lists to store results
train_mape_scores_mean = []
train_mape_scores_std = []
test_mape_scores_mean = []
test_mape_scores_std = []

# Loop over different depths
for feature in tqdm(max_features):
    # Define your estimator with the current max_depth
    rf_estimator = RandomForestRegressor(
        n_estimators=70,
        max_depth=25,
        min_samples_split=2,
        max_features=feature,
        criterion='squared_error',
        random_state=42,
        n_jobs=-1
    )

    # Build a Pipeline (you can add more steps if needed)
    pipeline = Pipeline(steps=[('model', rf_estimator)])

    # Call the function to perform manual CV and get MAPE metrics
    train_mape_mean, train_mape_std, test_mape_mean, test_mape_std = manual_cv_score(
        X=X_train_preprocessed,
        y=y_train,
        cv=cv,
        model=pipeline
    )

    # Store results
    train_mape_scores_mean.append(train_mape_mean)
    train_mape_scores_std.append(train_mape_std)
    test_mape_scores_mean.append(test_mape_mean)
    test_mape_scores_std.append(test_mape_std)

    # Print progress
    print(
        f"Max features: {feature}, "
        f"Train MAPE: {train_mape_mean:.2f}% (±{train_mape_std:.2f}), "
        f"Test MAPE: {test_mape_mean:.2f}% (±{test_mape_std:.2f})"
    )


In [ ]:
# Plot the MAPE scores
plt.figure(figsize=(8, 5))
plt.errorbar(feature, train_mape_scores_mean, yerr=train_mape_scores_std, label='Train MAPE', marker='o', capsize=5)
plt.errorbar(feature, test_mape_scores_mean, yerr=test_mape_scores_std, label='Test MAPE', marker='o', capsize=5)
plt.xlabel('Max Features')
plt.ylabel('MAPE (%)')
plt.title('Train vs Test MAPE Scores for Random Forest')
plt.legend()
plt.grid(True)
plt.show()


# Hyperparameter tuning: Min Samples Leaf

In [ ]:
min_samples_leafs = [1, 2, 3, 4, 5]

cv = KFold(n_splits=5, shuffle=True, random_state=42)

# Prepare lists to store results
train_mape_scores_mean = []
train_mape_scores_std = []
test_mape_scores_mean = []
test_mape_scores_std = []

# Loop over different depths
for leafs in tqdm(min_samples_leafs):
    # Define your estimator with the current max_depth
    rf_estimator = RandomForestRegressor(
        n_estimators=70,
        max_depth=25,
        min_samples_split=2,
        min_samples_leaf=leafs,
        max_features=1,
        criterion='squared_error',
        random_state=42,
        n_jobs=-1
    )

    # Build a Pipeline (you can add more steps if needed)
    pipeline = Pipeline(steps=[('model', rf_estimator)])

    # Call the function to perform manual CV and get MAPE metrics
    train_mape_mean, train_mape_std, test_mape_mean, test_mape_std = manual_cv_score(
        X=X_train_preprocessed,
        y=y_train,
        cv=cv,
        model=pipeline
    )

    # Store results
    train_mape_scores_mean.append(train_mape_mean)
    train_mape_scores_std.append(train_mape_std)
    test_mape_scores_mean.append(test_mape_mean)
    test_mape_scores_std.append(test_mape_std)

    # Print progress
    print(
        f"Max leafs: {leafs}, "
        f"Train MAPE: {train_mape_mean:.2f}% (±{train_mape_std:.2f}), "
        f"Test MAPE: {test_mape_mean:.2f}% (±{test_mape_std:.2f})"
    )

In [ ]:
# Plot the MAPE scores
plt.figure(figsize=(8, 5))
plt.errorbar(leafs, train_mape_scores_mean, yerr=train_mape_scores_std, label='Train MAPE', marker='o', capsize=5)
plt.errorbar(leafs, test_mape_scores_mean, yerr=test_mape_scores_std, label='Test MAPE', marker='o', capsize=5)
plt.xlabel('Max Features')
plt.ylabel('MAPE (%)')
plt.title('Train vs Test MAPE Scores for Random Forest')
plt.legend()
plt.grid(True)
plt.show()
